In [17]:
!pip install dlib


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\ASUS\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [18]:
pip show dlib

Name: dlib
Version: 20.0.0
Summary: A toolkit for making real world machine learning and data analysis applications
Home-page: https://github.com/davisking/dlib
Author: Davis King
Author-email: davis@dlib.net
License: Boost Software License
Location: C:\Users\ASUS\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages
Requires: 
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [19]:
import os
import cv2
import numpy as np
import dlib

In [20]:
import torch
import torch.nn as nn
from torchvision.models import mobilenet_v2
from torch.nn import functional as F

class Eye_model(nn.Module):
    def __init__(self, num_classes=2):
        super(Eye_model, self).__init__()

        # Load pretrained MobilenetV2 model
        self.base_model = mobilenet_v2(pretrained=True)
        out_features = self.base_model.classifier[1].out_features
        
        # Add custom layers
        self.head = nn.Sequential(
            nn.Flatten(),
            nn.Linear(out_features, 64),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(64, num_classes),
        )
        
        # Freeze base model layers
        for param in self.base_model.parameters():
            param.requires_grad = False

    def forward(self, inp):
        x = self.base_model(inp)
        if isinstance(x, tuple):
            x = x[0]
        return self.head(x)

model = Eye_model(num_classes=2)

In [21]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("D:\CSES\Year2\IoT\Project\shape_predictor_68_face_landmarks.dat")
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Load weights
model.load_state_dict(torch.load(r"D:\CSES\Year2\IoT\Project\checkpoint.pt", map_location=torch.device('cpu')))

C:\Users\ASUS\AppData\Local\Temp\ipykernel_7916\2926370483.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(r"D:\CSES\Year2\IoT\Project\c

<All keys matched successfully>

In [26]:
import cv2
import numpy as np
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import transforms

def extract_eyes_opencv_enhanced(img, gray):
  eye_images = []
  faces = face_cascade.detectMultiScale(gray, 1.1, 5, minSize=(100,100))
  
  for (x,y,w,h) in faces:
      roi_gray = gray[y:y+int(h*0.6), x:x+w]  # Only upper face region
      roi_color = img[y:y+int(h*0.6), x:x+w]
      
      # More sensitive parameters for eyes
      eyes = eye_cascade.detectMultiScale(
          roi_gray,
          scaleFactor=1.05,
          minNeighbors=5,
          minSize=(30,30),
          flags=cv2.CASCADE_SCALE_IMAGE
      )
      
      # Sort by area and take largest two detections
      eyes = sorted(eyes, key=lambda e: e[2]*e[3], reverse=True)[:2]
      
      for (ex,ey,ew,eh) in eyes:
          eye_img = roi_color[ey:ey+eh, ex:ex+ew]
          if eye_img.size > 0:
              eye_images.append(eye_img)
  
  return eye_images

# def extract_eyes_dlib_fast(img, gray):
#   eye_images = []
#   faces = detector(gray, 0)  # Faster with upsample=0
  
#   for face in faces:
#       # Get only the eye landmarks (skip others)
#       shape = predictor(gray, face)
      
#       # Left eye (points 36-41)
#       lx1 = shape.part(36).x
#       lx2 = shape.part(39).x
#       ly1 = min(shape.part(i).y for i in range(36, 42))
#       ly2 = max(shape.part(i).y for i in range(36, 42))
#       left_eye = img[ly1:ly2, lx1:lx2]
      
#       # Right eye (points 42-47)
#       rx1 = shape.part(42).x
#       rx2 = shape.part(45).x
#       ry1 = min(shape.part(i).y for i in range(42, 48))
#       ry2 = max(shape.part(i).y for i in range(42, 48))
#       right_eye = img[ry1:ry2, rx1:rx2]
      
#       if left_eye.size > 0: eye_images.append(left_eye)
#       if right_eye.size > 0: eye_images.append(right_eye)
  
#   return eye_images

def preprocess_eye(eye_img):
  """Prepare eye image for model input"""
  transform = transforms.Compose([
      transforms.ToPILImage(),
      transforms.Resize((64, 64)),
      transforms.Grayscale(num_output_channels=3),  # Convert to 3-channel if model expects RGB
      transforms.ToTensor(),
  ])
  return transform(eye_img)

def get_facial_landmarks(img, gray):
    faces = detector(gray)
    landmarks_list = []
    for face in faces:
        landmarks = predictor(gray, face)
        landmarks_list.append([(landmarks.part(i).x, landmarks.part(i).y) for i in range(68)])
    return landmarks_list

def display_output(img, predicted_class, landmarks_list=None):
  height, width, _ = img.shape
  font = cv2.FONT_HERSHEY_SIMPLEX
  font_scale = 2
  thickness = 3
  color = (0, 0, 255)  # Red

  text = str(predicted_class)
  (text_width, text_height), _ = cv2.getTextSize(text, font, font_scale, thickness)

  # Bottom-right corner position (10 pixels padding)
  x = width - text_width - 10
  y = height - 10
  cv2.putText(img, text, (x, y), font, font_scale, color, thickness)

  # Draw facial landmarks if available
  if landmarks_list:
      for landmarks in landmarks_list:
          for (x, y) in landmarks:
              cv2.circle(img, (x, y), 2, (0, 255, 0), -1)

  cv2.imshow("output", img)
  return img

def save_image(img, pred_class, count, f_name):
  if pred_class is not None:
    cv2.imwrite('out/{}_frame-{:03d}.jpg'.format(pred_class, count), img)
  else:
    cv2.imwrite('{}/frame-{:03d}.jpg'.format(f_name, count), img)
  count += 1
  return count

def define_labels():
  labels = {
      0 : 'aware',
      1 : 'drowsy',
      
    }
  return labels

def recognize():
    labels = define_labels()
    cam = cv2.VideoCapture(0)
    out_count = 1
    thresh_count = 1
    inp_count = 1

    preprocess = transforms.Compose([
      transforms.ToPILImage(),
      transforms.Resize((64, 64)),
      transforms.ToTensor(),
    ])

    model.eval()
    with torch.no_grad():
      while True:
          ret, img = cam.read()
          if not ret:
            break
          img = cv2.flip(img, 1)
          img_copy = img.copy()

          # Face detection
          gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
          faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(60, 60))

          for (x, y, w, h) in faces:
            cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)

          # Get facial landmarks with dlib
          landmarks_list = get_facial_landmarks(img, gray)

          # Get eye images
          eye_images = extract_eyes_opencv_enhanced(img, gray)
          
          predicted_class = None
          if len(eye_images) >= 2:  # At least both eyes detected
            # Process each eye and average predictions
            preds = []
            for eye in eye_images[:2]:  # Use only first two eyes if multiple faces
                eye_tensor = preprocess_eye(eye).unsqueeze(0)
                pred = model(eye_tensor)
                preds.append(pred)
            
            # Average predictions from both eyes
            combined_pred = torch.mean(torch.stack(preds), dim=0)
            predicted_class = labels[combined_pred.argmax().item()]
          elif len(eye_images) == 1:
            eye_tensor = preprocess_eye(eye_images[0]).unsqueeze(0)
            pred = model(eye_tensor)
            predicted_class = labels[pred.argmax().item()]

          ret_img = display_output(img, predicted_class, landmarks_list)
          # cv2.imshow("thresh", masked_frame)

          if cv2.waitKey(1) == ord('s'):
            inp_count = save_image(img_copy, None, inp_count, 'inp')
            out_count = save_image(ret_img, predicted_class, out_count, 'out')
          if cv2.waitKey(1) == ord('q'):
            break

    cam.release()
    cv2.destroyAllWindows()

recognize()